<a href="https://colab.research.google.com/github/antonychackotc/framework-machine-learning/blob/main/framework_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# create a streamlit application based

# upload a mulitple files and allow multiple format eg:- csv jason xcell etc
# and analyse file

# suggest me the which column is best for features colum and target column
# if that file have no target column (unlabeled means) display unsupervised learning

# if unsupervised learning
# recomend which algorithm sute for this file
# eg: kmeans clustering and explain why, or dbscan clustering and explain why
# and manual select option also give to select feature column and algorithm a
# and show time of loading time and alow

# else: supervised learning
# that dataset shows supervised learning
# recommend which algorithm sute for this file
# show classification and regression
# show continous and descret
# if regression
# if continous
# use - linear regression,polumomial regression,ridge regression and lasso regression, svm regression
# if discret
# use - decession tree regression, random forest regression, boosting regression,navebayes,knn regression, svm regression
# and
# if classify
# display - binary or multiclass
# if binary use - logistic regression, svm classifier
# else : multiclass use - decession tree classifier, random forest classifier, boosting classifier, knn classifier, svm classifier

# show recomendation of suitable algorithm - all algorithm manual select option - show accuracy with time

# show metrix

# show balanced or unbalanced dataset

# show precession, recall, f1 score, accuracy show both balanced or unbalanced dataset




In [ ]:
%%writefile app1.py

Writing app1.py


In [ ]:
!pip install -q streamlit
!pip install -q pyngrok
!pip install -q localtunnel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.9 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement localtunnel (from versions: none)
ERROR: No matching distribution found for localtunnel


# **Machine learning**

In [ ]:
%%writefile app1.py

import streamlit as st
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.svm import SVC, SVR
from sklearn.cluster import KMeans, DBSCAN
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, mean_squared_error, confusion_matrix)
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler, LabelEncoder
from io import StringIO
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import RFE
import time
from sklearn.pipeline import make_pipeline
import io
from sklearn.metrics import silhouette_score
from sklearn.cluster import AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.metrics import mean_squared_error, r2_score




# Application title
st.title("Machine Learning")
st.subheader("File Analysis and Algorithm Recommendation")
st.text("try to upload file without outliers")

# Tabs for application workflow

tabs = st.tabs([
    "Upload Files",
    "Dataset Analysis",
    "Supervised Learning",
    "Download Train and Test Datasets with Results",
    "Future Prediction",
    "Unsupervised Learning",
    "Data Quality Insights"
])



# Tab 1: File Upload
with tabs[0]:
    st.header("Upload Your Files")
    uploaded_files = st.file_uploader(
        "Upload multiple files (CSV, Excel, JSON)",
        type=["csv", "xlsx", "json"],
        accept_multiple_files=True
    )
    datasets = {}
    if uploaded_files:
        for uploaded_file in uploaded_files:
            file_type = uploaded_file.name.split(".")[-1].lower()
            try:
                if file_type == "csv":
                    data = pd.read_csv(uploaded_file)
                elif file_type in ["xls", "xlsx"]:
                    data = pd.read_excel(uploaded_file)
                elif file_type == "json":
                    data = pd.read_json(uploaded_file)
                else:
                    st.error(f"Unsupported file format: {file_type}")
                    continue

                datasets[uploaded_file.name] = data
                st.write(f"Preview of **{uploaded_file.name}**")
                st.write(data.head())

            except Exception as e:
                st.error(f"Error processing {uploaded_file.name}: {str(e)}")

# Dataset selection
selected_dataset = st.sidebar.selectbox(
    "Select a dataset to analyze",
    options=list(datasets.keys()) if datasets else [],
    help="Choose the dataset you uploaded for further analysis."
)

if selected_dataset:
    data = datasets[selected_dataset]

# Tab 2: Dataset Analysis
with tabs[1]:
    if selected_dataset:
        st.header(f"Analysis for {selected_dataset}")
        st.write("Shape of dataset:", data.shape)

        # Identify numeric and categorical columns
        numeric_cols = data.select_dtypes(include=["number"]).columns.tolist()
        categorical_cols = data.select_dtypes(include=["object", "category"]).columns.tolist()
        st.write("Numeric Columns:", numeric_cols)
        st.write("Categorical Columns:", categorical_cols)

        # Missing values
        missing_values = data.isnull().sum()
        st.write("Missing Values:")
        st.write(missing_values[missing_values > 0])

        # Outliers detection
        st.subheader("Outlier Detection (based on numeric columns):")
        for col in numeric_cols:
            q1 = data[col].quantile(0.25)
            q3 = data[col].quantile(0.75)
            iqr = q3 - q1
            outliers = data[(data[col] < (q1 - 1.5 * iqr)) | (data[col] > (q3 + 1.5 * iqr))]
            st.write(f"{col}: {len(outliers)} outliers")

        # Encoding recommendations
        if categorical_cols:
            st.subheader("Columns requiring encoding:")
            for col in categorical_cols:
                unique_values = data[col].nunique()
                encoding_type = "Ordinal" if unique_values < 10 else "Nominal"
                st.write(f"{col}: {encoding_type} encoding")

        # Scaling recommendations
        st.subheader("Columns requiring feature scaling (numeric only):")
        for col in numeric_cols:
            if data[col].max() - data[col].min() > 1000:
                st.write(col)



import numpy as np

from sklearn.metrics import (
    mean_squared_error,
    accuracy_score,
    classification_report,
    confusion_matrix,
)
import seaborn as sns


# Tab 3: Supervised Learning
with tabs[2]:
    # Ensure dataset is loaded
    if selected_dataset:
        st.header("Supervised Learning Recommendations")

        # Target column selection
        target_col = st.selectbox("Select the target column:", [None] + list(data.columns))
        if target_col:
            y = data[target_col]
            unique_values = y.nunique()

            # Determine if the target is continuous or discrete
            if y.dtype in [np.int64, np.float64]:
                if unique_values > 20:
                    inferred_task_type = "Regression"
                    target_status = "Continuous"
                else:
                    inferred_task_type = "Classification"
                    target_status = "Discrete"
            else:
                inferred_task_type = "Classification"
                target_status = "Discrete"

            # Display inferred target properties
            st.success(f"**Target Column Data Type:** {target_status}")
            st.success(f"**Inferred Task Type:** {inferred_task_type}")

            # Check for balance in classification tasks
            if inferred_task_type == "Classification":
                class_distribution = y.value_counts(normalize=True)
                max_class_proportion = class_distribution.max()
                balance_status = "Balanced" if max_class_proportion <= 0.7 else "Unbalanced"
                st.info(f"**Dataset Balance Status:** {balance_status}")
                st.write("**Class Distribution:")
                st.bar_chart(class_distribution)

                st.info(
    "Oversampling: - Add Extra Data\n"
    "1) Adds new samples of the minority class\n"
    "2) Used when there is not enough data\n"
    "3) Can be effective in small datasets\n"
    "4) Can lead to overfitting\n"
    "5) A popular technique is SMOTE (Synthetic Minority Over-sampling Technique)"

                )
                st.warning("Oversampling:\n"
                "Oversampling can create artificial class distributions that are not representative of the real world"
                )
                st.info(
    "Undersampling: - Remove Extra Data\n"
    "1)  Adds new samples of the majority class\n"
    "2) Used when there is too much data\n"
    "3) Can be effective in large datasets\n"
    "4) Can lead to loss of information\n"
    "5) Common methods include cluster centroids and Tomek links"
)
                st.warning("Undersampling:\n"
                "Undersampling can lead to biased models because it can cause loss of information. "
                )



            # Manual override for task type
            manual_task_type = st.radio(
                "Select the problem type manually:",
                options=["Classification", "Regression"],
                index=0 if inferred_task_type == "Classification" else 1,
            )
            manual_data_type = st.radio(
                "Select the target type manually:",
                options=["Continuous", "Discrete"],
                index=0 if target_status == "Continuous" else 1,
            )

            # Display suggested algorithms based on manual selection
            if manual_task_type == "Regression":
                st.write("**You selected Regression.**")
                st.write("**Suggested Regression Algorithms:**")
                if manual_data_type == "Continuous":
                    st.write("- Linear Regression")
                    st.write("- Polynomial Regression")
                    st.write("- Ridge Regression")
                    st.write("- Lasso Regression")
                    st.write("- SVM Regressor")
                else:
                    st.write("- Decision Tree Regressor")
                    st.write("- Random Forest Regressor")
                    st.write("- Boosting Regressor (e.g., XGBoost)")
                    st.write("- KNN Regressor")
                    st.write("- SVM Regressor")

            elif manual_task_type == "Classification":
                st.write("**You selected Classification.**")
                if unique_values == 2:
                    st.success("**Suggested Binary Classification Algorithms:**")
                    st.write("- Logistic Regression")
                    st.write("- SVM Classifier")
                else:
                    st.success("**Suggested Multiclass Classification Algorithms:**")
                    st.write("- Decision Tree Classifier")
                    st.write("- Random Forest Classifier")
                    st.write("- Boosting Classifier (e.g., XGBoost)")
                    st.write("- KNN Classifier")
                    st.write("- Naive Bayes Classifier")

            # Feature selection
            features = st.multiselect("Select feature columns", options=[col for col in data.columns if col != target_col])
            if features:
                X = data[features]
                y = data[target_col]

                # Map binary target column for classification
                if manual_task_type == "Classification" and unique_values == 2:
                    y = y.map({y.unique()[0]: 0, y.unique()[1]: 1})

                # Train-test split
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

                # Algorithm selection
                algorithm = st.selectbox(
                    "Select Algorithm",
                    [
                        "Linear Regression", "Polynomial Regression", "Ridge Regression", "Lasso Regression",
                        "ElasticNet Regression", "SVM Regression", "Decision Tree Regressor",
                        "Random Forest Regressor", "Boosting Regressor", "KNN Regressor"
                    ] if manual_task_type == 'Regression' else [
                        "Logistic Regression", "SVM Classifier", "Decision Tree Classifier",
                        "Random Forest Classifier", "Boosting Classifier", "KNN Classifier",
                        "Naive Bayes Classifier"
                    ]
                )

                # Initialize model
                model = None
                if algorithm == "Linear Regression":
                    model = LinearRegression()
                elif algorithm == "Polynomial Regression":
                    model = make_pipeline(PolynomialFeatures(degree=2), LinearRegression())
                elif algorithm == "Ridge Regression":
                    model = Ridge()
                elif algorithm == "Lasso Regression":
                    model = Lasso()
                elif algorithm == "Decision Tree Regressor":
                    model = DecisionTreeRegressor()
                elif algorithm == "Random Forest Regressor":
                    model = RandomForestRegressor()
                elif algorithm == "Boosting Regressor":
                    model = XGBRegressor()
                elif algorithm == "KNN Regressor":
                    model = KNeighborsRegressor()
                elif algorithm == "Logistic Regression":
                    model = LogisticRegression()
                elif algorithm == "SVM Classifier":
                    model = SVC()
                elif algorithm == "Decision Tree Classifier":
                    model = DecisionTreeClassifier()
                elif algorithm == "Random Forest Classifier":
                    model = RandomForestClassifier()
                elif algorithm == "Boosting Classifier":
                    model = XGBClassifier()
                elif algorithm == "KNN Classifier":
                    model = KNeighborsClassifier()
                elif algorithm == "Naive Bayes Classifier":
                    model = GaussianNB()

                # Train and evaluate the model
                start_time = time.time()
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)
                end_time = time.time()

                st.write(f"Training Time: {end_time - start_time:.2f} seconds")

                if manual_task_type == "Classification":
                    st.write(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
                    st.write(f"Precision: {precision_score(y_test, y_pred, average='macro'):.4f}")
                    st.write(f"Recall: {recall_score(y_test, y_pred, average='macro'):.4f}")
                    st.write(f"F1 Score: {f1_score(y_test, y_pred, average='macro'):.4f}")

                    st.info("Precision = TP/(TP + FP)")
                    st.info("Recall = TP/(TP + FN)")

                    # Display classification report as a table
                    report = classification_report(y_test, y_pred, output_dict=True)
                    report_df = pd.DataFrame(report).transpose()
                    st.table(report_df.style.format(precision=4))

                    # Plot Confusion Matrix
                    st.subheader("Confusion Matrix")
                    cm = confusion_matrix(y_test, y_pred)
                    fig, ax = plt.subplots()

                    # Annotate the confusion matrix with additional labels
                    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Predicted 0", "Predicted 1"], yticklabels=["Actual 0", "Actual 1"])
                    ax.set_xlabel("Predicted Labels")
                    ax.set_ylabel("True Labels")
                    ax.set_title("Confusion Matrix with Labels")

                    # Extract TP, TN, FP, FN values
                    tn, fp, fn, tp = cm.ravel()
                    st.write(f"**True Positive (TP): {tp}** - Predicted 1 and Actual 1")
                    st.write(f"**True Negative (TN): {tn}** - Predicted 0 and Actual 0")
                    st.write(f"**False Positive (FP): {fp}** - Predicted 1 but Actual 0")
                    st.write(f"**False Negative (FN): {fn}** - Predicted 0 but Actual 1")

                    # Display the heatmap
                    st.pyplot(fig)

                    st.markdown(
    """
    <b>True Positive</b>: The test correctly predicts a positive outcome when the actual outcome is positive.<br>
    <b>True Negative</b>: The test correctly predicts a negative outcome when the actual outcome is negative.<br>
    <b>False Positive</b>: The test incorrectly predicts a positive outcome when the actual outcome is negative.<br>
    <b>False Negative</b>: The test incorrectly predicts a negative outcome when the actual outcome is positive.
    """,
    unsafe_allow_html=True
)

                # Check if the task is classification
                    if manual_task_type == "Classification":
                        # Add a column to indicate TP, TN, FP, FN
                        result_labels = []
                        for true, pred in zip(y_test, y_pred):
                            if true == 1 and pred == 1:
                                result_labels.append("True Positive")
                            elif true == 0 and pred == 0:
                                result_labels.append("True Negative")
                            elif true == 0 and pred == 1:
                                result_labels.append("False Positive")
                            elif true == 1 and pred == 0:
                                result_labels.append("False Negative")

                        # Add this result to the test data for download
                        test_data_with_results = X_test.copy()
                        test_data_with_results[target_col] = y_test
                        test_data_with_results["Predicted"] = y_pred
                        test_data_with_results["Result"] = result_labels

                        # Check the shape of the data
                        print(test_data_with_results.shape)
                        print(test_data_with_results.head())

                        # Provide a download link
                        st.subheader("Download Updated Dataset")
                        buffer = io.BytesIO()
                        test_data_with_results.to_csv(buffer, index=False)
                        buffer.seek(0)
                        buffer.flush()  # Ensure everything is written to the buffer

                        st.write("Shape of dataset:", data.shape)
                        st.success("downloaded data is test data")
                        st.download_button(
                            label="Download Dataset with Results (CSV)",
                            data=buffer,
                            file_name="updated_dataset_with_results.csv",
                            mime="text/csv",
                        )


                elif manual_task_type == "Regression":
                    mse = mean_squared_error(y_test, y_pred)
                    st.write(f"Mean Squared Error: {mse:.4f}")

                    # Calculate R² score
                    r2 = r2_score(y_test, y_pred)
                    st.write(f"R² Score: {r2:.4f}")

                    st.write("MSE for Regression Tasks")
                    st.write("The range of values for mean squared error (MSE) is zero to infinity (\(0\le \text{MSE}<\infty \)). A lower MSE value indicates a better model. ")
                    st.write("The square root of MSE is called root mean square error (RMSE)")
                    st.write("the Mean Square Error (MSE) is a crucial metric for evaluating the performance of predictive models. It measures the average squared difference between the predicted and the actual target values within a dataset.")
        else:
            st.warning("Please select a target column to proceed.")

# Tab 4: Download Datasets with Results
with tabs[3]:
    st.header("Download Train and Test Datasets with Results")


    # Ensure features and target column are selected
    if features and target_col:
        if manual_task_type == "Classification":
            # Generate results for the train dataset
            train_result_labels = []
            for true, pred in zip(y_train, model.predict(X_train)):
                if true == 1 and pred == 1:
                    train_result_labels.append("True Positive")
                elif true == 0 and pred == 0:
                    train_result_labels.append("True Negative")
                elif true == 0 and pred == 1:
                    train_result_labels.append("False Positive")
                elif true == 1 and pred == 0:
                    train_result_labels.append("False Negative")

            train_data_with_results = X_train.copy()
            train_data_with_results[target_col] = y_train.values
            train_data_with_results["Predicted"] = model.predict(X_train)
            train_data_with_results["Result"] = train_result_labels

            # Generate results for the test dataset
            test_result_labels = []
            for true, pred in zip(y_test, y_pred):
                if true == 1 and pred == 1:
                    test_result_labels.append("True Positive")
                elif true == 0 and pred == 0:
                    test_result_labels.append("True Negative")
                elif true == 0 and pred == 1:
                    test_result_labels.append("False Positive")
                elif true == 1 and pred == 0:
                    test_result_labels.append("False Negative")

            test_data_with_results = X_test.copy()
            test_data_with_results[target_col] = y_test.values
            test_data_with_results["Predicted"] = y_pred
            test_data_with_results["Result"] = test_result_labels

            # Provide download links for train and test datasets
            st.subheader("Download Train Dataset with Results")
            train_buffer = io.BytesIO()
            train_data_with_results.to_csv(train_buffer, index=False)
            train_buffer.seek(0)
            st.download_button(
                label="Download Train Dataset with Results (CSV)",
                data=train_buffer,
                file_name="train_dataset_with_results.csv",
                mime="text/csv",
            )

            st.subheader("Download Test Dataset with Results")
            test_buffer = io.BytesIO()
            test_data_with_results.to_csv(test_buffer, index=False)
            test_buffer.seek(0)
            st.download_button(
                label="Download Test Dataset with Results (CSV)",
                data=test_buffer,
                file_name="test_dataset_with_results.csv",
                mime="text/csv",
            )

        elif manual_task_type == "Regression":
            st.warning("This feature is only available for Classification tasks.")
    else:
        st.warning("Please select features and a target column in the previous tab to proceed.")


#future prediction
with tabs[4]:
    if selected_dataset:
        st.header("Future Prediction")

        # Ensure model is trained and features are available
        if 'model' in locals() and model is not None:
            st.write("**Please input values for the following features to make a prediction:**")

            # Get feature names (assuming the model has been trained with 'X')
            model_features = X.columns

            # Create a dictionary to hold the user input
            user_input = {}

            # Create input fields for each feature
            for feature in model_features:
                user_input[feature] = st.text_input(f"Enter value for {feature}:")

            # When the user presses the button to make a prediction
            if st.button("Make Prediction"):
                try:
                    # Check if all the inputs are filled and valid
                    input_values = []
                    for feature in model_features:
                        value = user_input[feature]
                        # Convert to float and append to input_values
                        input_values.append(float(value))  # Could raise ValueError if the input is not a valid number

                    # Ensure the input is in the correct shape (1, n_features)
                    input_values = np.array(input_values).reshape(1, -1)

                    # Make the prediction using the trained model
                    prediction = model.predict(input_values)

                    # Show the prediction result
                    st.write(f"**Prediction Result:** {prediction[0]}")

                except ValueError:
                    st.error("Please enter valid numeric values for all features.")
                except Exception as e:
                    st.error(f"An error occurred: {str(e)}")
        else:
            st.warning("Model is not trained yet. Please train the model first.")



# Tab 4: Unsupervised Learning
with tabs[5]:
    if selected_dataset:
        st.header("Unsupervised Learning Recommendations")

        target_col = st.selectbox("Do you have a target column?", options=["No Target (Unlabeled)", "Yes"], index=0)

        if target_col == "No Target (Unlabeled)":
            st.write("Recommended Algorithms:")
            st.write("- **KMeans Clustering:** Centroid-based clustering for spherical clusters.")
            st.write("- **DBSCAN Clustering:** Density-based clustering for arbitrary shapes and noisy data.")
            st.write("- **Agglomerative Clustering:** Hierarchical clustering suitable for flexible group shapes.")
            st.write("- **Gaussian Mixture Models (GMM):** Probabilistic clustering assuming Gaussian distribution.")

            # Select features for clustering
            features = st.multiselect("Select feature columns for clustering", options=data.columns, default=numeric_cols)

            if features:
                st.write("Analyzing Dataset...")
                # Check for feature scaling requirement
                scaling_required = False
                for col in features:
                    if data[col].max() - data[col].min() > 1000 or abs(data[col].mean()) > 1000:
                        scaling_required = True
                        break

                if scaling_required:
                    st.write("⚠️ Feature scaling is recommended for the selected columns.")
                    scaler = StandardScaler()
                    data[features] = scaler.fit_transform(data[features])

                # Run clustering algorithms
                st.write("Running Clustering Algorithms...")
                start_time = time.time()

                clustering_results = {}
                silhouette_scores = {}


                # KMeans Clustering
                kmeans = KMeans(n_clusters=3, random_state=42).fit(data[features])
                data['KMeans_Labels'] = kmeans.labels_
                clustering_results['KMeans'] = kmeans.labels_
                silhouette_scores['KMeans'] = silhouette_score(data[features], kmeans.labels_)

                # DBSCAN Clustering
                dbscan = DBSCAN().fit(data[features])
                data['DBSCAN_Labels'] = dbscan.labels_
                clustering_results['DBSCAN'] = dbscan.labels_
                if len(set(dbscan.labels_)) > 1:  # Valid only if more than one cluster
                    silhouette_scores['DBSCAN'] = silhouette_score(data[features], dbscan.labels_)

                # Agglomerative Clustering
                agglomerative = AgglomerativeClustering(n_clusters=3).fit(data[features])
                data['Agglomerative_Labels'] = agglomerative.labels_
                clustering_results['Agglomerative'] = agglomerative.labels_
                silhouette_scores['Agglomerative'] = silhouette_score(data[features], agglomerative.labels_)

                # Gaussian Mixture Model (GMM)
                gmm = GaussianMixture(n_components=3, random_state=42).fit(data[features])
                gmm_labels = gmm.predict(data[features])
                data['GMM_Labels'] = gmm_labels
                clustering_results['GMM'] = gmm_labels
                silhouette_scores['GMM'] = silhouette_score(data[features], gmm_labels)

                end_time = time.time()
                st.write(f"Clustering completed in {round(end_time - start_time, 2)} seconds")

                # Display results and recommendations
                st.write("### Clustering Results and Metrics")
                for method, score in silhouette_scores.items():
                    st.write(f"**{method}:** Silhouette Score = {score:.2f}")

                # Recommend the best clustering method
                best_method = max(silhouette_scores, key=silhouette_scores.get)
                st.write(f"### Recommended Clustering Method: {best_method} (Highest Silhouette Score)")

                st.write("coefﬁcient is between [-1, 1]. A score of 1 denotes the best, meaning that the data point i is very compact within the cluster it belongs to and far away from the other clusters. The worst value is -1. Values near 0 denote overlapping")


                # Visualization of clustering
                if len(features) >= 2:
                    st.subheader("Clustering Visualizations")
                    for method, labels in clustering_results.items():
                        st.write(f"**{method} Clustering:**")
                        fig, ax = plt.subplots()
                        scatter = ax.scatter(data[features[0]], data[features[1]], c=labels, cmap='viridis', alpha=0.6)
                        ax.set_xlabel(features[0])
                        ax.set_ylabel(features[1])
                        st.pyplot(fig)

                # Download the clustering-based dataset
                st.subheader("Download Clustered Dataset")
                csv = data.to_csv(index=False)
                st.download_button(label="Download CSV", data=csv, file_name="clustered_dataset.csv", mime="text/csv")

                # Explain clustering methods
                st.subheader("Clustering Methods Explained")
                st.write("- **KMeans Clustering:** Assigns points to clusters based on proximity to centroids. Suitable for spherical clusters.")
                st.write("- **DBSCAN Clustering:** Groups points based on density. Handles noise and arbitrary cluster shapes well.")
                st.write("- **Agglomerative Clustering:** Performs hierarchical clustering. Flexible for various data shapes.")
                st.write("- **Gaussian Mixture Models (GMM):** Assumes clusters follow Gaussian distributions.")

        else:
            st.write("Supervised learning recommendations will be available on another tab.")

# Tab 5: Data Quality Insights
with tabs[6]:
    if selected_dataset:
        st.header("Data Quality Insights")

        # Missing values
        missing_values = data.isnull().sum()

        # Outliers detection
        outliers_count = 0
        for col in numeric_cols:
            # Calculate the IQR (Interquartile Range)
            q1 = data[col].quantile(0.25)
            q3 = data[col].quantile(0.75)
            iqr = q3 - q1

            # Find the outliers based on the IQR rule
            outliers = ((data[col] < (q1 - 1.5 * iqr)) | (data[col] > (q3 + 1.5 * iqr)))
            outliers_count += outliers.sum()

        st.write(f"Missing values: {missing_values.sum()} values across {len(missing_values[missing_values > 0])} columns")
        st.write(f"Outliers detected in {outliers_count} data points")

        # Visualize missing values
        st.subheader("Visualizing Missing Values")
        fig, ax = plt.subplots(figsize=(10, 5))
        sns.heatmap(data.isnull(), cbar=False, cmap="viridis", ax=ax)
        st.pyplot(fig)

        # Visualize outliers
        st.subheader("Visualizing Outliers")
        for col in numeric_cols:
            fig, ax = plt.subplots(figsize=(10, 5))
            sns.boxplot(data[col], ax=ax)
            ax.set_title(f"Outliers in {col}")
            st.pyplot(fig)

        # Feature Scaling Recommendation
        st.subheader("Feature Scaling")
        st.write("Feature scaling recommendations for numeric columns:")
        scaler = StandardScaler()
        scaled_data = pd.DataFrame(scaler.fit_transform(data[numeric_cols]), columns=numeric_cols)
        st.write(scaled_data.describe().T)

        # Encoding Recommendations
        st.subheader("Encoding Recommendations")
        st.write("Ordinal and Nominal encoding recommendations for categorical columns:")
        for col in categorical_cols:
            if data[col].nunique() <= 10:
                st.write(f"Column '{col}' can be encoded using **Ordinal Encoding**.")
            else:
                st.write(f"Column '{col}' can be encoded using **One-Hot Encoding**.")

        # Summary of Insights
        st.subheader("Summary of Data Quality Insights")
        st.write(f"- **Missing Values:** {missing_values.sum()} values in {missing_values[missing_values > 0].shape[0]} columns.")
        st.write(f"- **Outliers:** Detected in {outliers_count} data points across numeric columns.")
        st.write("- **Feature Scaling:** Recommended for numeric columns with a wide range.")
        st.write("- **Encoding:** Recommendations provided for categorical columns.")




# Final Notes
st.sidebar.info(
    """
    **Instructions:**
    - Upload your datasets in CSV, Excel, or JSON format.
    - Navigate through the tabs to explore dataset insights, supervised/unsupervised learning recommendations, and data quality insights.
    """
)

st.sidebar.success("Developed by [ TC.Antony - Data Scientist] ")



Overwriting app1.py


In [ ]:
from pyngrok import ngrok

# Replace 'YOUR_AUTHTOKEN' with your actual ngrok authtoken
ngrok.set_auth_token("2rI2XurhgC2fxlYDtteHntWpCJf_5b1kDx2SLmwgq8GukDEyc")

# Run the Streamlit app in the background
!streamlit run app1.py &>/dev/null&

# Create a public URL using ngrok
try:
    public_url = ngrok.connect(8501)
    print(f"Streamlit app is running at {public_url}")
except Exception as e:
    print(f"Error: {e}")
    print("Trying to run with localtunnel")
    !streamlit run app1.py &>/content/logs.txt & npx localtunnel --port 8501

Streamlit app is running at NgrokTunnel: "https://5388-34-106-125-219.ngrok-free.app" -> "http://localhost:8501"
